In [1]:
from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [2]:
target_image_path = '/content/abc.jpg'
style_image_path = '/content/def.jpg'

In [3]:
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width*img_height / height)

In [4]:
import numpy as np
from keras.applications import vgg19

In [5]:
def preprocess_image(image_path):
  img = load_img(image_path,target_size = (img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img,axis=0)
  img = vgg19.preprocess_input(img)
  return img

In [23]:
def deprocess_image(x):
  x[:,:,0]+=103.939
  x[:,:,1]+=116.779
  x[:,:,2]+=123.68
  x=x[:,:,::-1]
  x=np.clip(x,0,255).astype('uint8')
  return x

In [7]:
from keras import backend as K

In [8]:
target_image = K.constant(preprocess_image(target_image_path))
style_image = K.constant(preprocess_image(style_image_path))
combination_image = K.placeholder((1,img_height,img_width,3))


In [9]:
input_tensor = K.concatenate([target_image,style_image,combination_image],axis=0)

In [10]:
model = vgg19.VGG19(input_tensor = input_tensor,
                    weights='imagenet',
                    include_top=False)

In [11]:
print('Model loaded')

Model loaded


In [12]:
def content_loss(base, combination):
  return K.sum(K.square(combination-base))

In [13]:
def gram_matrix(x):
  assert K.ndim(x)==3
  features = K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
  gram = K.dot(features, K.transpose(features))
  return gram

In [14]:
def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height*img_width
  return K.sum(K.square(S-C))/(4. *(channels**2)*(size**2))

In [15]:
def total_variational_loss(x):
  a = K.square(
      x[:,:img_height-1,:img_width-1,:]-
      x[:,1:,:img_width-1,:])
  b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, :img_height - 1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))
  

In [16]:
output_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [17]:
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025
loss = K.variable(0.)
layer_features = output_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2,:,:,:]
loss=loss+content_weight * content_loss(base=target_image_features,combination=combination_features)


In [18]:
for layer_name in style_layers:
  layer_features = output_dict[layer_name]
  style_features = layer_features[1,:,:,:]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style=style_features, combination=combination_features)
  loss =loss + (style_weight / len(style_layers)) * sl

In [19]:
loss = loss+total_variation_weight * total_variational_loss(combination_image)

In [20]:
grads = K.gradients(loss = loss,variables=combination_image)[0]
fetch_loss_and_grads = K.function([combination_image],[loss,grads])

In [21]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape([1, img_height, img_width, 3])
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


evaluator = Evaluator()

In [24]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'my_result'
iterations = 20

x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                     x,
                                     fprime=evaluator.grads,
                                     maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 3800852700.0
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 459s
Start of iteration 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 1566881700.0
Image saved as my_result_at_iteration_1.png
Iteration 1 completed in 460s
Start of iteration 2
Current loss value: 1002585000.0
Image saved as my_result_at_iteration_2.png
Iteration 2 completed in 462s
Start of iteration 3
Current loss value: 805453440.0
Image saved as my_result_at_iteration_3.png
Iteration 3 completed in 465s
Start of iteration 4
Current loss value: 679068000.0
Image saved as my_result_at_iteration_4.png
Iteration 4 completed in 459s
Start of iteration 5
Current loss value: 600773440.0
Image saved as my_result_at_iteration_5.png
Iteration 5 completed in 463s
Start of iteration 6
Current loss value: 548959600.0
Image saved as my_result_at_iteration_6.png
Iteration 6 completed in 465s
Start of iteration 7
Current loss value: 503390820.0
Image saved as my_result_at_iteration_7.png
Iteration 7 completed in 461s
Start of iteration 8
Current loss value: 463343260.0
Image saved as my_result_at_iteration_8.png
Iteration 8 completed in 458s
Sta